In [107]:
# Reading in excel spreadsheet

import pandas as pd
import os 

data = 'ETO use.xlsx'

# read in use data
use_df = pd.read_excel(data, sheet_name='eto_use_alt')

# read in users data
CL_df = pd.read_excel(data, sheet_name='CL Codes')

In [108]:
### INPUTS ###

### date range to be invoiced
date_min = '2024-04-01'
date_max = '2024-04-30'

# Create Directory for saving invoice
directory = "2024.04 Invoicing"

month = 'April'
year = '2024'



In [109]:
# drop spaces from account lists
use_df['Account'] = use_df['Account'].str.replace(' ', '')

# Split the 'Account' column into a list of accounts
use_df['Account'] = use_df['Account'].str.split(',')

# Use explode() to expand the accounts into separate rows while keeping the associated date
use_df_expanded = use_df.explode('Account')

# determine number of accounts during a given run
date_counts = use_df_expanded.groupby('Date').size().reset_index(name='Account_Count')

# Merge the count information back into the original DataFrame
use_df_expanded = use_df_expanded.merge(date_counts, on='Date')

# assign % charge based off the counts
percent_charge = []
for i, count in use_df_expanded.iterrows():
    percent_charge.append(1 / use_df_expanded['Account_Count'][i])

use_df_expanded['Percent_Charge'] = percent_charge
use_df_expanded = use_df_expanded.drop(columns=('Account_Count'))

# Print the resulting DataFrame
# use_df_expanded

In [110]:
# create subset df for period of interest

#### Date range of interest must be updated ####

period_of_interest_df = use_df_expanded[(use_df_expanded['Date'] >= date_min) & (use_df_expanded['Date'] <= date_max)]

In [111]:
# creating new dataframe to total uses and charge

charges_df = pd.DataFrame()
charges_df['Account'] = use_df_expanded['Account'].unique()
charges_df['PI'] = 0
charges_df['Number Uses'] = 0
charges_df['Total ($)'] = 0

charges_df = charges_df.sort_values(by='Account')


# fill in PI by referencng CL codes df
PI = []

for i, row in CL_df.iterrows():
    for j, event in charges_df.iterrows():
        if CL_df['CL_code'][i] == charges_df['Account'][j]:
            PI.append(CL_df['PI'][i])

charges_df['PI'] = PI
charges_df = charges_df.reset_index().drop(columns='index')
# charges_df



In [112]:
# filling in charges df for period of interest

for i, row in charges_df.iterrows():
    owed = []
    num_uses = []
    
    for j, rows in period_of_interest_df.iterrows():
        if charges_df['Account'][i] == period_of_interest_df['Account'][j]:
            num_uses.append(period_of_interest_df['Percent_Charge'][j])

        charges_df['Number Uses'][i] = sum(num_uses)
        
for i, row in charges_df.iterrows():
    charges_df['Total ($)'][i] = charges_df['Number Uses'][i]*40

charges_df = charges_df.round(2)

# charges_df
    

/var/folders/ky/zl0049892b138kmk5t49jgjr0000gp/T/ipykernel_11196/2703409962.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charges_df['Total ($)'][i] = charges_df['Number Uses'][i]*40
/var/folders/ky/zl0049892b138kmk5t49jgjr0000gp/T/ipykernel_11196/2703409962.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charges_df['Total ($)'][i] = charges_df['Number Uses'][i]*40
/var/folders/ky/zl0049892b138kmk5t49jgjr0000gp/T/ipykernel_11196/2703409962.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [113]:
# drop official use row

charges_df = charges_df[charges_df['Account'] != 'CL000']

# only keep rows with uses/charges
charges_df = charges_df[charges_df['Number Uses'] != 0]

charges_df

,Account,PI,Number Uses,Total ($)


In [114]:
### File save path needs to be updated for each invoice ###

# Create Directory 
directory = directory
  
# Parent Directory path 
parent_dir = "../eto_billing"
  
# Path 
path = os.path.join(parent_dir, directory) 

if not os.path.exists(path):
    os.makedirs(path) 

charges_df.to_excel(f"2024.04 Invoicing/ethylene_oxide_invoice_{month}_{year}.xlsx", index=False)